In [5]:
from pyspark.sql.types import StringType
from pyspark.sql.functions import udf
#import org.apache.spark.sql.functions.monotonicallyIncreasingId
from pyspark import SparkContext
from pyspark import SparkConf, SparkContext ,SQLContext,Row,HiveContext
import pyspark.sql.functions as func
from pyspark.sql.functions import lit
from datetime import datetime
from pyspark.sql.functions import col
from functools import reduce
from pyspark.sql import DataFrame
#import org.apache.spark.sql.expressions._
from pyspark import HiveContext
from pyspark.sql.types import *
from pyspark.sql import Row, functions as F
from pyspark.sql.functions import row_number
from pyspark.sql.window import Window
from pyspark.sql.functions import broadcast
from datetime import datetime
from datetime import timedelta
import hashlib
import sys

In [ ]:
#variables
eff_close_dt = '3099-12-31'
current_bus_dt='2018-02-18'
eff_flag_curr = 'Y'
eff_flag_non_curr = 'N'
eff_start_date_hist = 'Today'
eff_start_date_delta = 'Tomorrow' ## 1 day lead
dt=datetime.now()
load_tm=dt.strftime('%Y-%m-%d %H:%M:%S')

In [ ]:
#Day 0 Load
day0_data=spark.read.csv("/Users/sandip/Data/spark/SCD2/store.csv",header='true',inferSchema='true')
#Surrogate key generation
day0_data_with_sk=day0_data.select("Store_ID").withColumn("store_id_sk",F.row_number().over(Window.partitionBy(lit(1)).orderBy(lit(1))))
#add SCD2 column
day0_sk_df = day0_data.join(broadcast(day0_data_with_sk) ,'Store_ID','inner').withColumn('eff_start_date',lit(current_bus_dt)).withColumn('eff_end_date',lit(eff_close_dt)).withColumn('load_tm',lit(load_tm)).withColumn('current_flag',lit(eff_flag_curr)).withColumn('is_delta',lit('N'))
#Save data in parquet format with partition on store id
day0_sk_df.write.mode('overwrite').partitionBy("Store_ID").format("parquet").save("/Users/sandip/Data/spark/SCD2/output/")

In [ ]:
#Day 1 Load 
day1=spark.read.csv("/Users/sandip/Data/spark/SCD2/Day1.csv",header='true',inferSchema='true')

#Rename columns of dataframe
day1=day1.withColumnRenamed('Store_ID','delta_Store_ID').withColumnRenamed('Location','delta_Location').withColumnRenamed('Value','delta_Value')

#read existing data 
history= spark.read.parquet("/Users/sandip/Data/spark/SCD2/output/")
#Get maximum number from sk column to increment for day 1 load
max_account_sk_id_rdd_list = history.agg({'store_id_sk': 'max'}).rdd.map(list)
for line in max_account_sk_id_rdd_list.collect():
    max_store_id_sk=line[0]
#Capture modified records from day1 load
cdc_change_df = day1.join(history ,(history.Store_ID == day1.delta_Store_ID) ,'left_outer' ).where((history['Value'] != day1['delta_Value']) & (history['current_flag'] == lit(eff_flag_curr)))
#Capture new records from day1 load
cdc_new_df = day1.join(history ,(day1.delta_Store_ID == history.Store_ID) ,'left_outer' ).where(history['Value'].isNull())
#Mearge both
cdc_all_record_df = cdc_new_df.unionAll(cdc_change_df)
#Add sk cloumn and other
delta_sk_lkp_df = cdc_all_record_df.select('delta_Store_ID').withColumn('delta_Store_sk_ID', F.row_number().over(Window.partitionBy(lit(1)).orderBy(lit(1))))
cdc_all_record_sk_df = cdc_all_record_df.join(broadcast(delta_sk_lkp_df) ,'delta_Store_ID', 'inner').withColumn('max_store_sk_id',lit(max_store_id_sk)).withColumn('delta_Store_sk_ID',lit(max_store_id_sk + delta_sk_lkp_df.delta_Store_sk_ID)).withColumn('delta_eff_start_date',lit(current_bus_dt)).withColumn('delta_eff_end_date',lit(eff_close_dt)).withColumn('delta_load_tm',lit(load_tm)).withColumn('delta_current_flag',lit(eff_flag_curr)).withColumn('is_delta',lit('Y'))

#Adding temp flag to capture delta
cdc_all_record_sk_ld_df = cdc_all_record_sk_df.select('delta_Location','delta_Value','delta_Store_sk_ID','delta_eff_start_date','delta_eff_end_date','delta_load_tm','delta_current_flag','is_delta','delta_Store_ID')
cdc_change_df_id=cdc_change_df.select("Location","Value","store_id_sk","eff_start_date","eff_end_date","load_tm","current_flag","Store_ID").withColumn('is_delta',lit('N'))
cdc_change_df_id=cdc_change_df_id.select("Location","Value","store_id_sk","eff_start_date","eff_end_date","load_tm","current_flag",'is_delta',"Store_ID")
hist_cdc_change_df = cdc_change_df_id.withColumn('eff_end_date',lit(load_tm)).withColumn('current_flag',lit(eff_flag_non_curr))

#mearge final dataframe
final=hist_cdc_change_df.unionAll(cdc_all_record_sk_ld_df)

#final.write.mode('append').partitionBy("Store_ID").format("parquet").save("/Users/sandip/Data/spark/SCD2/output/")

In [ ]:
#Push new records 
x_final=final.where(final['current_flag'] == lit(eff_flag_curr))
x_final.write.mode('append').partitionBy("Store_ID").format("parquet").save("/Users/sandip/Data/spark/SCD2/output/")

In [ ]:
#Modify existing records for maintaing SCD2
store_list = hist_cdc_change_df.select('Store_ID').rdd.map(list)
y=store_list.collect()
for x in range(len(y)):
    history= spark.read.parquet("/Users/sandip/Data/spark/SCD2/output/")
    updated_df = history.where((history["Store_ID"]==lit(y[x][0])) & (history['current_flag'] == lit(eff_flag_non_curr))& (history['is_delta'] == lit("N")))
    updated_new_df = history.where((history["Store_ID"]==lit(y[x][0])) & (history['current_flag'] == lit(eff_flag_curr))& (history['is_delta'] == lit("Y")))
    updated_df.count()
    new_df=hist_cdc_change_df.where(hist_cdc_change_df["Store_ID"]==lit(y[x][0]))
    new_df.count()
    final_df=updated_df.unionAll(updated_new_df).unionAll(new_df)
    final_df.write.mode('overwrite').format("parquet").save("/Users/sandip/Data/spark/SCD2/Temp/")
    x_df=spark.read.parquet("/Users/sandip/Data/spark/SCD2/Temp/")
    x_df.write.mode('overwrite').format("parquet").save("/Users/sandip/Data/spark/SCD2/output/Store_ID="+y[x][0])    